# Classification Comparison Analysis

This notebook compares two classification methods against human-labeled ground truth for the HBP content modernization project.

## Overview
- **Ground Truth**: Human-labeled data with Y/N flags for removal
- **Experiment 1 (Copilot Studio)**: LLM responses with KEEP/REMOVE recommendations parsed from `CP_FULL_RESPONSE`
- **Experiment 2 (Alternate)**: Y/N flags from another approach

## Metrics Computed
- Confusion Matrix (TP, TN, FP, FN)
- Precision, Recall, F1 Score
- Accuracy
- Comparison visualizations

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics import (
    confusion_matrix, 
    classification_report, 
    precision_score, 
    recall_score, 
    f1_score, 
    accuracy_score
)
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options
pd.set_option('display.max_colwidth', 100)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

## 1. Configuration

**Update these paths and column names to match your files.**

In [ ]:
# =============================================================================
# FILE PATHS - UPDATE THESE TO YOUR LOCAL FILE LOCATIONS
# =============================================================================

GROUND_TRUTH_FILE = "ground_truth.xlsx"  # Human-labeled ground truth
COPILOT_EXPERIMENT_FILE = "copilot_results.xlsx"  # LLM output with CP_FULL_RESPONSE
ALTERNATE_EXPERIMENT_FILE = "alternate_results.xlsx"  # Second experiment with Y/N

# =============================================================================
# COLUMN NAMES - UPDATE THESE TO MATCH YOUR SPREADSHEETS
# =============================================================================

# Ground truth file columns
GT_ID_COLUMN = "core_product_id"  # or whatever your ID column is called
GT_FLAG_COLUMN = "flag for removal"  # Y = remove, N = keep

# Copilot experiment file columns
CP_ID_COLUMN = "core_product_id"  # ID column in copilot results
CP_RESPONSE_COLUMN = "CP_FULL_RESPONSE"  # Column with full LLM response to parse

# Alternate experiment file columns  
ALT_ID_COLUMN = "core_product_id"  # ID column in alternate results
ALT_FLAG_COLUMN = "flag for removal"  # Y = remove, N = keep

## 2. Load Data

In [ ]:
# Load ground truth
print("Loading ground truth...")
df_ground_truth = pd.read_excel(GROUND_TRUTH_FILE)
print(f"  Loaded {len(df_ground_truth)} rows")
print(f"  Columns: {list(df_ground_truth.columns)}")

# Load Copilot experiment results
print("\nLoading Copilot experiment results...")
df_copilot = pd.read_excel(COPILOT_EXPERIMENT_FILE)
print(f"  Loaded {len(df_copilot)} rows")
print(f"  Columns: {list(df_copilot.columns)}")

# Load alternate experiment results
print("\nLoading alternate experiment results...")
df_alternate = pd.read_excel(ALTERNATE_EXPERIMENT_FILE)
print(f"  Loaded {len(df_alternate)} rows")
print(f"  Columns: {list(df_alternate.columns)}")

## 3. Parse LLM Responses

Extract KEEP/REMOVE from the `CP_FULL_RESPONSE` column.

In [ ]:
def parse_recommendation(response_text):
    """
    Parse the LLM response to extract KEEP or REMOVE recommendation.
    
    Looks for patterns like:
    - **Overall Recommendation:** KEEP
    - **Overall Recommendation:** REMOVE
    - Overall Recommendation: KEEP
    - etc.
    
    Returns: 'KEEP', 'REMOVE', or 'UNPARSEABLE'
    """
    if pd.isna(response_text) or not isinstance(response_text, str):
        return 'UNPARSEABLE'
    
    # Normalize the text
    text = response_text.upper()
    
    # Pattern 1: Look for "Overall Recommendation:" followed by KEEP or REMOVE
    # Handle various markdown formatting (**, *, etc.)
    pattern = r'\*{0,2}OVERALL\s+RECOMMENDATION:?\*{0,2}\s*\*{0,2}(KEEP|REMOVE)\*{0,2}'
    match = re.search(pattern, text)
    
    if match:
        return match.group(1)
    
    # Pattern 2: Look for "Recommendation:" followed by KEEP or REMOVE
    pattern2 = r'RECOMMENDATION:?\s*\*{0,2}(KEEP|REMOVE)\*{0,2}'
    match2 = re.search(pattern2, text)
    
    if match2:
        return match2.group(1)
    
    # Pattern 3: Just look for KEEP or REMOVE at the end of the text
    # (sometimes LLMs put the final answer at the very end)
    if text.strip().endswith('KEEP'):
        return 'KEEP'
    if text.strip().endswith('REMOVE'):
        return 'REMOVE'
    
    return 'UNPARSEABLE'


# Test the parser on a sample
test_cases = [
    "Based on my analysis... **Overall Recommendation:** KEEP**",
    "The content is outdated. **Overall Recommendation:** REMOVE",
    "Overall Recommendation: KEEP",
    "This article should be flagged. Recommendation: REMOVE",
    "Some text without a clear recommendation",
    None,
]

print("Testing parser:")
for tc in test_cases:
    result = parse_recommendation(tc)
    display_text = str(tc)[:60] + "..." if tc and len(str(tc)) > 60 else tc
    print(f"  '{display_text}' -> {result}")

In [ ]:
# Apply parser to Copilot results
df_copilot['parsed_recommendation'] = df_copilot[CP_RESPONSE_COLUMN].apply(parse_recommendation)

# Check parsing results
print("Parsing results summary:")
print(df_copilot['parsed_recommendation'].value_counts())

# Show any unparseable rows for investigation
unparseable = df_copilot[df_copilot['parsed_recommendation'] == 'UNPARSEABLE']
if len(unparseable) > 0:
    print(f"\n⚠️  {len(unparseable)} rows could not be parsed. Sample responses:")
    for idx, row in unparseable.head(3).iterrows():
        print(f"\n  ID: {row[CP_ID_COLUMN]}")
        response = str(row[CP_RESPONSE_COLUMN])[:500]
        print(f"  Response: {response}...")

## 4. Standardize Labels

Convert all labels to a common format:
- `1` = REMOVE (flagged for removal)
- `0` = KEEP (not flagged)

In [ ]:
def standardize_label(value):
    """
    Convert various label formats to binary:
    - Y, REMOVE, 1, True -> 1 (remove)
    - N, KEEP, 0, False -> 0 (keep)
    """
    if pd.isna(value):
        return np.nan
    
    val = str(value).upper().strip()
    
    if val in ['Y', 'YES', 'REMOVE', '1', 'TRUE']:
        return 1
    elif val in ['N', 'NO', 'KEEP', '0', 'FALSE']:
        return 0
    else:
        return np.nan


# Standardize ground truth
df_ground_truth['gt_label'] = df_ground_truth[GT_FLAG_COLUMN].apply(standardize_label)

# Standardize Copilot results
df_copilot['cp_label'] = df_copilot['parsed_recommendation'].apply(standardize_label)

# Standardize alternate experiment
df_alternate['alt_label'] = df_alternate[ALT_FLAG_COLUMN].apply(standardize_label)

print("Ground Truth distribution:")
print(df_ground_truth['gt_label'].value_counts(dropna=False))

print("\nCopilot predictions distribution:")
print(df_copilot['cp_label'].value_counts(dropna=False))

print("\nAlternate experiment distribution:")
print(df_alternate['alt_label'].value_counts(dropna=False))

## 5. Merge Datasets

Join all results on the article ID.

In [ ]:
# Prepare dataframes for merging (select only needed columns)
gt_for_merge = df_ground_truth[[GT_ID_COLUMN, 'gt_label']].copy()
gt_for_merge = gt_for_merge.rename(columns={GT_ID_COLUMN: 'article_id'})

cp_for_merge = df_copilot[[CP_ID_COLUMN, 'cp_label', 'parsed_recommendation']].copy()
cp_for_merge = cp_for_merge.rename(columns={CP_ID_COLUMN: 'article_id'})

alt_for_merge = df_alternate[[ALT_ID_COLUMN, 'alt_label']].copy()
alt_for_merge = alt_for_merge.rename(columns={ALT_ID_COLUMN: 'article_id'})

# Merge all datasets
df_merged = gt_for_merge.merge(cp_for_merge, on='article_id', how='outer')
df_merged = df_merged.merge(alt_for_merge, on='article_id', how='outer')

print(f"Merged dataset: {len(df_merged)} rows")
print(f"\nData availability:")
print(f"  - Ground truth labels: {df_merged['gt_label'].notna().sum()}")
print(f"  - Copilot predictions: {df_merged['cp_label'].notna().sum()}")
print(f"  - Alternate predictions: {df_merged['alt_label'].notna().sum()}")

# Show sample
df_merged.head(10)

## 6. Calculate Classification Metrics

In [ ]:
def calculate_metrics(y_true, y_pred, method_name):
    """
    Calculate and display classification metrics.
    
    Positive class (1) = REMOVE
    Negative class (0) = KEEP
    """
    # Filter out any NaN values
    mask = (~pd.isna(y_true)) & (~pd.isna(y_pred))
    y_true_clean = y_true[mask].astype(int)
    y_pred_clean = y_pred[mask].astype(int)
    
    n_samples = len(y_true_clean)
    
    if n_samples == 0:
        print(f"\n❌ {method_name}: No valid samples to compare!")
        return None
    
    # Calculate confusion matrix
    cm = confusion_matrix(y_true_clean, y_pred_clean)
    
    # Extract values (handling cases where not all classes are present)
    if cm.shape == (2, 2):
        tn, fp, fn, tp = cm.ravel()
    else:
        # Handle edge case where only one class is present
        tn = fp = fn = tp = 0
        if 0 in y_true_clean.values and 0 in y_pred_clean.values:
            tn = ((y_true_clean == 0) & (y_pred_clean == 0)).sum()
        if 0 in y_true_clean.values and 1 in y_pred_clean.values:
            fp = ((y_true_clean == 0) & (y_pred_clean == 1)).sum()
        if 1 in y_true_clean.values and 0 in y_pred_clean.values:
            fn = ((y_true_clean == 1) & (y_pred_clean == 0)).sum()
        if 1 in y_true_clean.values and 1 in y_pred_clean.values:
            tp = ((y_true_clean == 1) & (y_pred_clean == 1)).sum()
    
    # Calculate metrics
    accuracy = accuracy_score(y_true_clean, y_pred_clean)
    precision = precision_score(y_true_clean, y_pred_clean, zero_division=0)
    recall = recall_score(y_true_clean, y_pred_clean, zero_division=0)
    f1 = f1_score(y_true_clean, y_pred_clean, zero_division=0)
    
    # Display results
    print(f"\n{'='*60}")
    print(f"📊 {method_name.upper()} RESULTS")
    print(f"{'='*60}")
    print(f"\nSamples evaluated: {n_samples}")
    print(f"\n📋 Confusion Matrix:")
    print(f"                    Predicted")
    print(f"                 KEEP    REMOVE")
    print(f"  Actual KEEP     {tn:4d}     {fp:4d}   (TN, FP)")
    print(f"  Actual REMOVE   {fn:4d}     {tp:4d}   (FN, TP)")
    print(f"\n📈 Metrics:")
    print(f"  Accuracy:  {accuracy:.4f} ({accuracy*100:.1f}%)")
    print(f"  Precision: {precision:.4f} (of predicted REMOVE, how many were correct)")
    print(f"  Recall:    {recall:.4f} (of actual REMOVE, how many were found)")
    print(f"  F1 Score:  {f1:.4f} (harmonic mean of precision and recall)")
    
    # Return metrics as dict for comparison
    return {
        'method': method_name,
        'n_samples': n_samples,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'confusion_matrix': cm
    }

In [ ]:
# Calculate metrics for Copilot experiment
copilot_metrics = calculate_metrics(
    df_merged['gt_label'], 
    df_merged['cp_label'], 
    'Copilot Studio'
)

In [ ]:
# Calculate metrics for Alternate experiment
alternate_metrics = calculate_metrics(
    df_merged['gt_label'], 
    df_merged['alt_label'], 
    'Alternate Method'
)

## 7. Side-by-Side Comparison

In [ ]:
# Create comparison table
if copilot_metrics and alternate_metrics:
    comparison_data = {
        'Metric': ['Samples', 'True Positives', 'True Negatives', 'False Positives', 
                   'False Negatives', 'Accuracy', 'Precision', 'Recall', 'F1 Score'],
        'Copilot Studio': [
            copilot_metrics['n_samples'],
            copilot_metrics['TP'],
            copilot_metrics['TN'],
            copilot_metrics['FP'],
            copilot_metrics['FN'],
            f"{copilot_metrics['accuracy']:.4f}",
            f"{copilot_metrics['precision']:.4f}",
            f"{copilot_metrics['recall']:.4f}",
            f"{copilot_metrics['f1']:.4f}"
        ],
        'Alternate Method': [
            alternate_metrics['n_samples'],
            alternate_metrics['TP'],
            alternate_metrics['TN'],
            alternate_metrics['FP'],
            alternate_metrics['FN'],
            f"{alternate_metrics['accuracy']:.4f}",
            f"{alternate_metrics['precision']:.4f}",
            f"{alternate_metrics['recall']:.4f}",
            f"{alternate_metrics['f1']:.4f}"
        ]
    }
    
    df_comparison = pd.DataFrame(comparison_data)
    print("\n" + "="*70)
    print("📊 SIDE-BY-SIDE COMPARISON")
    print("="*70)
    print(df_comparison.to_string(index=False))
else:
    print("Cannot create comparison - one or both methods have no valid data.")

## 8. Visualizations

In [ ]:
def plot_confusion_matrix(cm, title, ax):
    """Plot a confusion matrix heatmap."""
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
                xticklabels=['KEEP', 'REMOVE'],
                yticklabels=['KEEP', 'REMOVE'])
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')
    ax.set_title(title)


if copilot_metrics and alternate_metrics:
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    plot_confusion_matrix(copilot_metrics['confusion_matrix'], 
                          'Copilot Studio', axes[0])
    plot_confusion_matrix(alternate_metrics['confusion_matrix'], 
                          'Alternate Method', axes[1])
    
    plt.tight_layout()
    plt.savefig('confusion_matrices.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("\n💾 Saved: confusion_matrices.png")

In [ ]:
# Bar chart comparison of metrics
if copilot_metrics and alternate_metrics:
    metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
    copilot_values = [copilot_metrics['accuracy'], copilot_metrics['precision'], 
                      copilot_metrics['recall'], copilot_metrics['f1']]
    alternate_values = [alternate_metrics['accuracy'], alternate_metrics['precision'], 
                        alternate_metrics['recall'], alternate_metrics['f1']]
    
    x = np.arange(len(metrics_names))
    width = 0.35
    
    fig, ax = plt.subplots(figsize=(10, 6))
    bars1 = ax.bar(x - width/2, copilot_values, width, label='Copilot Studio', color='#2196F3')
    bars2 = ax.bar(x + width/2, alternate_values, width, label='Alternate Method', color='#FF9800')
    
    ax.set_ylabel('Score')
    ax.set_title('Classification Metrics Comparison')
    ax.set_xticks(x)
    ax.set_xticklabels(metrics_names)
    ax.legend()
    ax.set_ylim(0, 1.1)
    
    # Add value labels on bars
    for bar in bars1:
        height = bar.get_height()
        ax.annotate(f'{height:.2f}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3), textcoords="offset points",
                    ha='center', va='bottom', fontsize=9)
    for bar in bars2:
        height = bar.get_height()
        ax.annotate(f'{height:.2f}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3), textcoords="offset points",
                    ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.savefig('metrics_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("\n💾 Saved: metrics_comparison.png")

## 9. Error Analysis

Examine where the methods disagree with ground truth.

In [ ]:
# Find disagreements
df_analysis = df_merged.copy()

# Copilot errors
df_analysis['cp_correct'] = df_analysis['gt_label'] == df_analysis['cp_label']
df_analysis['cp_error_type'] = np.where(
    df_analysis['cp_label'].isna(), 'NO_PREDICTION',
    np.where(df_analysis['gt_label'].isna(), 'NO_GROUND_TRUTH',
    np.where(df_analysis['cp_correct'], 'CORRECT',
    np.where((df_analysis['gt_label'] == 0) & (df_analysis['cp_label'] == 1), 'FALSE_POSITIVE',
    np.where((df_analysis['gt_label'] == 1) & (df_analysis['cp_label'] == 0), 'FALSE_NEGATIVE',
    'OTHER')))))

# Alternate errors
df_analysis['alt_correct'] = df_analysis['gt_label'] == df_analysis['alt_label']
df_analysis['alt_error_type'] = np.where(
    df_analysis['alt_label'].isna(), 'NO_PREDICTION',
    np.where(df_analysis['gt_label'].isna(), 'NO_GROUND_TRUTH',
    np.where(df_analysis['alt_correct'], 'CORRECT',
    np.where((df_analysis['gt_label'] == 0) & (df_analysis['alt_label'] == 1), 'FALSE_POSITIVE',
    np.where((df_analysis['gt_label'] == 1) & (df_analysis['alt_label'] == 0), 'FALSE_NEGATIVE',
    'OTHER')))))

print("Copilot Error Analysis:")
print(df_analysis['cp_error_type'].value_counts())

print("\nAlternate Method Error Analysis:")
print(df_analysis['alt_error_type'].value_counts())

In [ ]:
# Show false positives (predicted REMOVE but should KEEP)
print("\n🔴 COPILOT FALSE POSITIVES (predicted REMOVE, actual KEEP):")
cp_fp = df_analysis[df_analysis['cp_error_type'] == 'FALSE_POSITIVE']
print(f"Count: {len(cp_fp)}")
if len(cp_fp) > 0:
    print(cp_fp[['article_id', 'gt_label', 'cp_label', 'parsed_recommendation']].head(10))

In [ ]:
# Show false negatives (predicted KEEP but should REMOVE)
print("\n🔴 COPILOT FALSE NEGATIVES (predicted KEEP, actual REMOVE):")
cp_fn = df_analysis[df_analysis['cp_error_type'] == 'FALSE_NEGATIVE']
print(f"Count: {len(cp_fn)}")
if len(cp_fn) > 0:
    print(cp_fn[['article_id', 'gt_label', 'cp_label', 'parsed_recommendation']].head(10))

## 10. Export Results

In [ ]:
# Export full analysis to Excel
output_file = 'classification_comparison_results.xlsx'

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    # Summary sheet
    if copilot_metrics and alternate_metrics:
        df_comparison.to_excel(writer, sheet_name='Summary', index=False)
    
    # Full merged data with analysis
    df_analysis.to_excel(writer, sheet_name='Full Analysis', index=False)
    
    # Copilot errors
    cp_errors = df_analysis[df_analysis['cp_error_type'].isin(['FALSE_POSITIVE', 'FALSE_NEGATIVE'])]
    cp_errors.to_excel(writer, sheet_name='Copilot Errors', index=False)
    
    # Alternate errors
    alt_errors = df_analysis[df_analysis['alt_error_type'].isin(['FALSE_POSITIVE', 'FALSE_NEGATIVE'])]
    alt_errors.to_excel(writer, sheet_name='Alternate Errors', index=False)

print(f"\n💾 Results exported to: {output_file}")

## 11. Summary & Recommendations

In [ ]:
if copilot_metrics and alternate_metrics:
    print("\n" + "="*70)
    print("📋 SUMMARY & RECOMMENDATIONS")
    print("="*70)
    
    # Determine winner for each metric
    metrics_to_compare = ['accuracy', 'precision', 'recall', 'f1']
    
    print("\n🏆 Best Performer by Metric:")
    for metric in metrics_to_compare:
        cp_val = copilot_metrics[metric]
        alt_val = alternate_metrics[metric]
        if cp_val > alt_val:
            winner = "Copilot Studio"
            diff = cp_val - alt_val
        elif alt_val > cp_val:
            winner = "Alternate Method"
            diff = alt_val - cp_val
        else:
            winner = "TIE"
            diff = 0
        print(f"  {metric.capitalize():12s}: {winner} (+{diff:.4f})")
    
    # Overall recommendation based on F1 (balanced metric)
    print("\n📌 Overall Recommendation:")
    if copilot_metrics['f1'] > alternate_metrics['f1']:
        print("  → Copilot Studio shows better overall performance (higher F1)")
    elif alternate_metrics['f1'] > copilot_metrics['f1']:
        print("  → Alternate Method shows better overall performance (higher F1)")
    else:
        print("  → Both methods perform equally (same F1)")
    
    # Note about recall importance for this use case
    print("\n💡 Note for HBP use case:")
    print("  - High RECALL is important if you want to catch most outdated content")
    print("    (minimize false negatives - items that should be removed but aren't flagged)")
    print("  - High PRECISION is important if you want to minimize manual review workload")
    print("    (minimize false positives - items flagged that shouldn't be)")